<a href="https://colab.research.google.com/github/kjt2010/cs152bots/blob/main/Geagea1_Twitter_Topic_Explorer_for_students_spring_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

## Imports and functions

In [ ]:
%%capture
%pip install twarc
%pip install twint
%pip install nest_asyncio
%pip install altair_saver
%pip install requests
!git clone https://github.com/DocNow/twarc
!apt install curl jq
!npm install vega-lite vega-cli canvas # for altair_saver

In [ ]:
%%capture
import os
import re
import json
import pandas as pd
import numpy as np
import altair as alt
import nest_asyncio
import twint
import datetime
import requests
import functools
from dateutil.parser import parse as date_parse
from os import path
from twarc import Twarc
from collections import Counter
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import Image
from IPython.core.display import HTML 
from altair_saver import save as altair_save

if 'google.colab' in str(get_ipython()):
    from googleapiclient.discovery import build
    from google.colab import files
    from google.colab import auth
    from google.colab import drive
    %load_ext google.colab.data_table

pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 20)
pd.options.display.float_format = '{:,.2f}'.format
alt.data_transformers.disable_max_rows()
nest_asyncio.apply()

In [ ]:
!npm install -g unshrtn
!nohup unshrtn start &

@functools.lru_cache()
def unshrtn(url):
  resp = requests.get("http://localhost:3000", params={"url": url}).json()
  for key in ["canonical", "long", "short"]:
    if key in resp:
      return (resp[key], resp)
  return (None, None)

def unshrtn_tweet_dict(tweet):
  # Note: do we also need to look at the retweeted status (if available), or
  # does the top-level `entities` list suffice?
  if "entities" in tweet and "urls" in tweet["entities"]:
    for url_obj in tweet["entities"]["urls"]:
      url, metadata = unshrtn(url_obj["url"])
      url_obj["unwound_url"] = url
      url_obj["unwound_url_meta"] = metadata

npm WARN deprecated request-promise@4.2.6: request-promise has been deprecated because it extends the now deprecated request package, see https://github.com/request/request/issues/3142
npm WARN deprecated request-promise-native@1.0.9: request-promise-native has been deprecated because it extends the now deprecated request package, see https://github.com/request/request/issues/3142
npm WARN deprecated request@2.88.2: request has been deprecated, see https://github.com/request/request/issues/3142
npm WARN deprecated har-validator@5.1.5: this library is no longer supported
npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
npm WARN deprecated formidable@1.2.6: Please upgrade to latest, formidable@v2 or formidable@v3! Check these notes: https://bit.ly/2ZEqIau
/tools/node/bin/unshrtn -> /tools/node/lib/node_modules/unshrtn/c

In [ ]:
BEGINNING_OF_TIME = "2006-03-20 00:00:00"
NOW = datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S')

def find_first_tweet(keyword, original_since = BEGINNING_OF_TIME, 
                     original_until = NOW, output_csv = 'out.csv', 
                     format = '%Y-%m-%d %H:%M:%S'): 

    # specify search time range 
    timediff = datetime.timedelta(days=3)

    since = original_since
    since_time = datetime.datetime.strptime(since, format)

    until_time = datetime.datetime.strptime(since, format) + timediff
    until = datetime.datetime.strftime(until_time, format)
    original_until_time = datetime.datetime.strptime(original_until, format)

    # specify how quickly we find the keyword 
    aggressive_constant = 1.3 
    constant = 1 
    hashtag = keyword
    last_fail = ""
    last_pull = False

    while True: 
        if path.exists(output_csv):
          os.remove(output_csv)

        c = twint.Config()
        c.Limit = 1
        c.Since = since 
        c.Until = until
        c.Search = hashtag
        c.Store_csv = True
        c.Output = output_csv
        twint.run.Search(c)

        if last_pull is True: 
          break 

        # if we found the tweet or searched all the time ranges 
        if os.path.exists(output_csv) or until_time >= original_until_time :
          out = pd.read_csv('out.csv')
          first_success = out['date'][out.shape[0]-1] + \
                            ' ' + out['time'][out.shape[0]-1]
          until_time = datetime.datetime.strptime(first_success, format)
          until = datetime.datetime.strftime(until_time, format)
          last_pull = True
        else:
          # search more aggressively in the next time range 
          last_pull = False
          timediff *= aggressive_constant * constant 
          since_time = until_time
          until_time = since_time + timediff  
          last_fail = since_time
          since = datetime.datetime.strftime(since_time, format)
          until = datetime.datetime.strftime(until_time, format)

In [ ]:
def present(name, chart):
  """Saves and displays an Altair chart"""
  altair_save(chart, name + ".png")
  display(chart)

## Twitter Keys

In [ ]:
# Your Twitter developer keys, consumer key means API key, consumer scret means API secret key

consumer_key = "lqfj1IGjdgwpvgo33xSWVyFFO"
consumer_secret = "pJWGwpserRG9mZwflohcAIjf4ZGIDugVh8YtXly9f6xVWXgHd7"
access_token = "1439719938764132355-9WAuRgtEoFD1DrBubt96TewZWdP6f4"
access_token_secret = "gmFThwJVXirPrXfWRxkb7I5PuNApxuZ3DeEBPfEa0hSgs"

In [ ]:
# Your Gnip API keys, if not using Twitter

gnip_username = ""
gnip_password = ""
gnip_account = "StanfordInternetObservatory"

In [ ]:
gnip = False
t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret, app_auth=True)

if gnip_username != "":
    g = Twarc(gnip_auth=True, gnip_username=gnip_username, gnip_password=gnip_password, gnip_account=gnip_account)
    gnip = True

# Tweet search

## Search query

*Return up to a week of tweets for a keyword or hashtag.*

https://developer.twitter.com/en/docs/twitter-api/v1/rules-and-filtering/overview/standard-operators

*Use double quotes (inside single quotes) for exact multi-word phrase match*

In [ ]:
# Number of tweets desired 
limit = 100

query = 'المقاومة_الاسلامية'

# You can specify from_date and to_date only if using Gnip. Remove the # from the `date_parse` examples
# to enable time boxing.
from_date = None
to_date = None
# from_date = date_parse("Jan 3, 2020")
# to_date = date_parse("Jan 6, 2020")

# A sensible name for the output files and buckets
# Letters, numbers and underscores only
output_name = "Geagea2_output"

In [ ]:
# Run search loop
f = open(output_name + '.jsonl', 'w')
bar = IntProgress(min=0, max=limit)
display(bar)

for tweet_idx, tweet in enumerate(g.premium_search(query, "gnip_fullarchive", "prod", from_date=from_date, to_date=to_date) if gnip else t.search(query)):
    print(json.dumps(tweet), file=f)
    if tweet_idx + 1 == limit:
      break
    # update progress bar    
    if (tweet_idx + 1) % 100 == 0:
      bar.value += 100
      
bar.value = limit
f.close()

IntProgress(value=0)

In [ ]:
# Unflatten JSON arrays
df = pd.json_normalize(pd.Series(open(output_name + '.jsonl').readlines()).apply(json.loads))

# Make timestamps sane
df["created_at"] = df["created_at"].astype('datetime64[ns]')
df["user.created_at"] = df["user.created_at"].astype('datetime64[ns]')

# Fix gnip fields
if gnip:
    df["full_text"] = df["text"]

### Basic account info and tweets

# Stats & Viz

### Highest engagement tweets

In [ ]:
# Sort by retweet or favorite in the interactive DF
df[["created_at", 
    "id_str",
    "user.screen_name", 
    "retweet_count", 
    "favorite_count",
    "full_text"]].sort_values('favorite_count', ascending=False)

,created_at,id_str,user.screen_name,retweet_count,favorite_count,full_text
77,2021-12-05 19:50:28,1467582161742090240,RLehbib,6,41,🦅 الناطق باسم حركة المقاومة #حماس يؤكد في ذكرى...
89,2021-12-05 19:43:50,1467580491360518144,Alaqsavoice_Brk,4,29,تصريح صحفي صادر عن الناطق باسم حركة المقاومة ا...
87,2021-12-05 19:45:20,1467580871565885445,ShehabAgency,4,24,تصريح صحفي صادر عن الناطق باسم حركة المقاومة ا...
96,2021-12-05 19:34:18,1467578095884509197,aboaskar233,4,11,سَجَّل المقاومة ..\nحدث في مثل هذا اليوم بتاري...
10,2021-12-05 22:32:30,1467622939780657152,Antiamerica7,2,4,ارشيف المقاومه ..\nحدث في مثل هذا اليوم بتاريخ...
12,2021-12-05 22:29:12,1467622108364775428,raji9919206,0,4,كل المجتمعات في العالم تقمصت النموذج الغربي في...
76,2021-12-05 19:50:41,1467582216662302720,Ananalnasrei,2,3,طائرات حزب الله؟\nالمقاومة الاسلامية تكشف النق...
90,2021-12-05 19:41:28,1467579895379382277,Alaqsavoice,0,2,تصريح صحفي صادر عن الناطق باسم حركة المقاومة ا...
20,2021-12-05 21:39:05,1467609494494625814,Mohamad12206088,0,1,هل تعلمون ان المقاومه الاسلاميه ستهزمكم. كما ه...
66,2021-12-05 19:57:41,1467583976680349696,hichem1088,0,0,@Ghost_Medhi @BouriyaM @AhmedDe93610794 @omar_...


### Noisiest users

In [ ]:
noise_df = df["user.screen_name"].value_counts()[:30].to_frame().reset_index()

# Rename for better labels
noise_df["screen_name"] = noise_df["index"]
noise_df["count"] = noise_df["user.screen_name"]

present(
    "noisiest_users",
    alt.Chart(noise_df).mark_bar().encode(y=alt.Y('screen_name', sort='-x'), x="count")
)

alt.Chart(...)

### Top Twitter clients

In [ ]:
client_df = df.source.apply(lambda x: re.sub('<[^<]+?>', '', x)).value_counts().to_frame().reset_index()

# Rename for better labels
client_df["client"] = client_df["index"]
client_df["count"] = client_df["source"]

present(
    "top_clients",
    alt.Chart(client_df).mark_bar().encode(y=alt.Y('client', sort='-x'), x=alt.X("count", scale=alt.Scale(type='log')))
)

WARN Cannot stack non-linear scale (log).
WARN Log scale domain includes zero: [0,69]


alt.Chart(...)

### Top hashtags

In [ ]:
min_hashtag_frequency = 1 # change to 1 to display all hashtags
max_hashtags_to_display = 10 # change to None to display all

top_hashtags = sum(df["entities.hashtags"], [])
top_hashtags = Counter([hashtag['text'] for hashtag in top_hashtags])
top_hashtags = Counter({hashtag: count for hashtag, count in top_hashtags.items() if count >= min_hashtag_frequency})

raw_hashtag_data = dict(top_hashtags.most_common(max_hashtags_to_display))
top_hashtags_df = pd.DataFrame.from_dict(raw_hashtag_data, orient="index", columns=["count"]).reset_index()

top_hashtags_df["hashtag"] = top_hashtags_df["index"]

present(
    "top_hashtags",
    alt.Chart(top_hashtags_df).mark_bar().encode(y=alt.Y('hashtag', sort='-x'), x=alt.X("count"))
)

alt.Chart(...)

### Highly mentioned users

In [ ]:
min_mentions_frequency = 1   # change to 1 to display all mentions
max_mentions_to_display = 20 # change to None to display all
top_mentioned_users = sum(df["entities.user_mentions"], [])
top_mentioned_users = Counter([user['screen_name'] for user in top_mentioned_users])
top_mentioned_users = Counter({user: count for user, count in top_mentioned_users.items() if count >= min_mentions_frequency})

top_mentioned_users = pd.Series(dict(top_mentioned_users.most_common(max_mentions_to_display)))
top_mentioned_users

mention_df = top_mentioned_users.to_frame().reset_index()

# Rename for better labels
noise_df["screen_name"] = noise_df["index"]
noise_df["count"] = noise_df["user.screen_name"]

present("most_mentioned_users", alt.Chart(noise_df).mark_bar().encode(y=alt.Y('screen_name', sort='-x'), x="count"))

alt.Chart(...)

### Languages

In [ ]:
present("languages", alt.Chart(df[['lang']]).mark_bar().encode(
    alt.X("lang", bin=False),
    y='count()',
))

alt.Chart(...)

### Account creation date

In [ ]:
# Zero out the time
tmpdf = df
tmpdf["user.created_at"] = tmpdf["user.created_at"].dt.normalize()

# Select unique
tmpdf.drop_duplicates(['user.id_str','created_at'])

present("created_date", alt.Chart(tmpdf[['user.created_at']]).mark_bar().encode(
    alt.X("[user.created_at]:T"),
    y='count()',
))

alt.Chart(...)

### Tweets over time

In [ ]:
# General activity
tmpdf["created_at"] = tmpdf["created_at"].dt.normalize()

present("tweets_over_time", alt.Chart(tmpdf[['created_at']]).mark_line().encode(
    alt.X("created_at:T"),
    y='count()',
))

alt.Chart(...)

### User timeseries

In [ ]:
noise_df = df["user.screen_name"].value_counts()[:10].to_frame().reset_index()
top_users_only = df[df["user.screen_name"].isin(noise_df["index"])]

grouped_df = (
    top_users_only.groupby(["created_at", "user.screen_name"])
    .size()
    .reset_index(name="count")
)

# Rename for better labels
grouped_df["user"] = grouped_df["user.screen_name"]

present(
    "user_timeseries",
    alt.Chart(grouped_df)
    .mark_line()
    .encode(x="created_at", y="count", color="user", strokeDash="user"),
)

alt.Chart(...)

### Hashtag timeseries

In [ ]:
expanded_hashtag_df = pd.DataFrame(
    {
        col: np.repeat(df[col].values, df["entities.hashtags"].str.len())
        for col in df.columns.drop("entities.hashtags")
    }
).assign(**{"entities.hashtags": np.concatenate(df["entities.hashtags"].values)})[
    df.columns
]

expanded_hashtag_df["hashtag"] = expanded_hashtag_df["entities.hashtags"].map(
    lambda k: k["text"]
)

top_hashtags = expanded_hashtag_df["hashtag"].value_counts()[:10].index

grouped_df = (
    expanded_hashtag_df[expanded_hashtag_df["hashtag"].isin(top_hashtags)]
    .groupby(["created_at", "hashtag"])
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)

present(
    "hashtag_timeseries",
    alt.Chart(grouped_df)
    .mark_line()
    .encode(x="created_at", y="count", color="hashtag", strokeDash="hashtag"),
)

alt.Chart(...)

### First use of hashtag

In [ ]:
# Reverse chronology
#find_first_tweet("#BLM")

## Gephi graphs

In [ ]:
# Make a Gephi user network
!twarc/utils/network.py {output_name}.jsonl user-network.gexf --users
files.download('user-network.gexf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Make a Gephi hashtag network
!twarc/utils/network.py {output_name}.jsonl hashtag-network.gexf --hashtags
files.download('hashtag-network.gexf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>